<a href="https://colab.research.google.com/github/DepartmentOfStatisticsPUE/bi-2021/blob/main/materialy-wyklady/bi_2021_05_17_rivers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

W tym skrypcie zastosujemy metodę opartą na pracy [Rivers, D. (2007). Sampling for web surveys, Proceedings of the Survey Research Methods
Section of the American Statistical Association, pp. 1–26.](https://static.texastribune.org/media/documents/Rivers_matching4.pdf). 

W tym ćwiczeniu porównamy dwa podejścia:

1. wykorzystując zmienne $\mathbf{X}$ do łączenia
2. wykorzystując wartości przewidywane $\hat{y}_i$ na podstawie modelu regresji logistycznej

Poszukiwania sąsiadów dokonamy wykorzystując metodę K-D tree dla przybliżonych sąsiadów oprogramowaną w pakiecie `RANN` (funkcja `nn2`), która opiera się na bibliotece [ANN w C++](http://www.cs.umd.edu/~mount/ANN/).

In [ ]:
install.packages("RANN")